# Claim 1.1 HS Improves Predictive performance of Decision Trees

In [22]:
# imports
%load_ext autoreload
%autoreload 2
import os
import sys

import matplotlib as mpl
from sklearn.model_selection import train_test_split
from sklearn.tree import export_text, DecisionTreeClassifier, DecisionTreeRegressor

import numpy as np
import pandas as pd

import imodels
from imodels.util.data_util import get_clean_dataset # this was used to get the datasets

from copy import deepcopy

# cross-validation of models
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

# timing algorithm execution
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# Repositories used for tests (location: paper_autors_repo/config/shrinkage/models.py) - tuki sem skopiral samo ker je lazje najti

DATASETS_CLASSIFICATION = [
    # classification datasets from original random forests paper
    # page 9: https://www.stat.berkeley.edu/~breiman/randomforest2001.pdf
    ("heart", "heart", 'imodels'),
    ("breast-cancer", "breast_cancer", 'imodels'), # this is the wrong breast-cancer dataset (https://new.openml.org/search?type=data&sort=runs&id=13&status=active)
    ("haberman", "haberman", 'imodels'),
    ("ionosphere", "ionosphere", 'pmlb'),
    ("diabetes", "diabetes", "pmlb"),
    ("german-credit", "german", "pmlb"),
    ("juvenile", "juvenile_clean", 'imodels'),
    ("recidivism", "compas_two_year_clean", 'imodels'),
]

DATASETS_REGRESSION = [
    ('friedman1', 'friedman1', 'synthetic'),
    ('friedman3', 'friedman3', 'synthetic'),
    ("diabetes-regr", "diabetes", 'sklearn'),
    # missing red-wine and geographical music added later
    ('abalone', '183', 'openml'),
    ("satellite-image", "294_satellite_image", 'pmlb'),
    ("california-housing", "california_housing", 'sklearn'),
]

In [24]:
tasks_classification = {}

# load datasets
for task in DATASETS_CLASSIFICATION:
    X, y, feature_names = get_clean_dataset(task[1], data_source = task[2])
    df = pd.DataFrame(X, columns=feature_names)
    df["label"] = y
    tasks_classification[task[0]] = df
    
tasks_regression = {}

# load datasets
for task in DATASETS_REGRESSION:
    X, y, feature_names = get_clean_dataset(task[1], data_source = task[2])
    df = pd.DataFrame(X, columns=feature_names)
    df["label"] = y
    tasks_regression[task[0]] = df
    
# add missing datasets (not present in code)
# red-wine dataset
wine = pd.read_csv("data/missing_data/winequality-red.csv", sep = ";")
wine = (wine-wine.mean())/wine.std()
wine.rename(columns = {'quality':'label'}, inplace = True)

tasks_regression["red-wine"] = wine
DATASETS_REGRESSION.append(("red-wine", "red-wine", "None"))

# geographical-music dataset (omitted for now since it is a 2D output dataset (RF should and can predict latitude/longitude))
# 116 & 117 are part of label
music = pd.read_csv("data/missing_data/geo-music-big.txt", header = None)

In [25]:
def leaf_count(dt):
    tree = dt.tree_
    num_leaves = 0

    for i in range(tree.node_count):
        if tree.children_left[i] > 0 or tree.children_right[i] > 0:
            num_leaves += 1

    return num_leaves

# fix-up alpha
def pick_alpha(X, y, number_of_leaves, dt_class):
    path = dt_class().cost_complexity_pruning_path(X, y)
    alphas = path["ccp_alphas"]

    leaf_cnts = []

    for alpha in alphas:
        mccp = dt_class(ccp_alpha=alpha).fit(X, y)
        leaf_cnts.append(np.abs(number_of_leaves - leaf_count(mccp)))
    idx = np.argmin(leaf_cnts)

    return alphas[idx]

"""
def pick_alpha(X, y, number_of_leaves, dt_class, scorer):
    path = dt_class(max_leaf_nodes=number_of_leaves).cost_complexity_pruning_path(X, y)
    alphas = path["ccp_alphas"]
    
    cv_scores = {}
    
    for alpha in alphas:
        hs_skf = StratifiedKFold(n_splits=3)
        cv_scores[alpha] = []
        for j, (cv_train_index, cv_val_index) in enumerate(hs_skf.split(X, y)):
            X_cv_train, y_cv_train = X[cv_train_index, :], y[cv_train_index]
            X_cv_val, y_cv_val = X[cv_val_index, :], y[cv_val_index]
            mccp = dt_class(ccp_alpha=alpha).fit(X_cv_train, y_cv_train)
            y_val_pred = mccp.predict(X_cv_val)
            cv_scores[alpha].append(scorer(y_cv_val, y_val_pred))
    
    cv_scores = {reg_param: np.mean(cv_scores[reg_param]) for reg_param in cv_scores.keys()}
    best_score = np.max([cv_scores[reg_param] for reg_param in cv_scores.keys()])
    best_param = [reg_param for reg_param in cv_scores.keys() if cv_scores[reg_param] == best_score][0]
    
    return best_param
"""

'\ndef pick_alpha(X, y, number_of_leaves, dt_class, scorer):\n    path = dt_class(max_leaf_nodes=number_of_leaves).cost_complexity_pruning_path(X, y)\n    alphas = path["ccp_alphas"]\n    \n    cv_scores = {}\n    \n    for alpha in alphas:\n        hs_skf = StratifiedKFold(n_splits=3)\n        cv_scores[alpha] = []\n        for j, (cv_train_index, cv_val_index) in enumerate(hs_skf.split(X, y)):\n            X_cv_train, y_cv_train = X[cv_train_index, :], y[cv_train_index]\n            X_cv_val, y_cv_val = X[cv_val_index, :], y[cv_val_index]\n            mccp = dt_class(ccp_alpha=alpha).fit(X_cv_train, y_cv_train)\n            y_val_pred = mccp.predict(X_cv_val)\n            cv_scores[alpha].append(scorer(y_cv_val, y_val_pred))\n    \n    cv_scores = {reg_param: np.mean(cv_scores[reg_param]) for reg_param in cv_scores.keys()}\n    best_score = np.max([cv_scores[reg_param] for reg_param in cv_scores.keys()])\n    best_param = [reg_param for reg_param in cv_scores.keys() if cv_scores[reg_

## 4.2 A) Classification

In [26]:
# number of leafs used in paper
num_of_leaves = [2, 4, 8, 12, 15, 20, 24, 28, 30, 32]
# reuglarization parameter
reg_hs = [0.1, 1.0, 10.0, 25.0, 50.0, 100.0]

# Potential problem HS appeared to be choosen via CV (hopefully they split the dataset before hand)
NUM_OF_BOOTSTRAP_SAMPS = 10
classification_results = pd.DataFrame(columns = ["task", "dataset", "boot_iter", "algorithm", "scoring", "n_leaves", "max_leaves", "regularization", "train_score", "test_score", \
                                                 "train_wall_time", "test_wall_time", "train_cpu_time", "test_cpu_time", "tunning_wall_time", "tunning_cpu_time"])

for task in DATASETS_CLASSIFICATION:
    for samp in range(NUM_OF_BOOTSTRAP_SAMPS):
        skf = StratifiedKFold(n_splits=3)
        X, y = np.array(tasks_classification[task[0]].drop("label", axis = 1)), np.array(tasks_classification[task[0]]["label"])
        for i, (train_index, test_index) in enumerate(skf.split(tasks_classification[task[0]], tasks_classification[task[0]]["label"])):
            print(f"Dataset: {task[0]}, Sample: {samp}, Fold {i}", end = "\r")

            X_train, y_train = X[train_index, :], y[train_index]
            X_test, y_test = X[test_index, :], y[test_index]

            for m in num_of_leaves:
                ### CART ###
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                m1 = DecisionTreeClassifier(max_leaf_nodes=m).fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_dt = m1.predict(X_train)
                y_test_pred_dt = m1.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["DT"],
                                                                            "scoring": ["AUC"],
                                                                            "n_leaves": [leaf_count(m1)],
                                                                            "max_leaves": [m],
                                                                            "regularization": ["None"],
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_dt)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_dt)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [None], 
                                                                            "tunning_cpu_time": [None]})])

                ### CART with CCP ###
                
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                best_alpha = pick_alpha(X_train, y_train, m, DecisionTreeClassifier)
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                mccp = DecisionTreeClassifier(ccp_alpha=best_alpha).fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_ccp = mccp.predict(X_train)
                y_test_pred_ccp = mccp.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["CCP"],
                                                                            "scoring": ["AUC"],
                                                                            "n_leaves": [leaf_count(mccp)],
                                                                            "max_leaves": [m],
                                                                            "regularization": [best_alpha],
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_ccp)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_ccp)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])

                ### C4.5 DT ###
                """
                mc45 = imodels.C45TreeClassifier().fit(X_train, y_train)
                y_train_pred_c45 = mc45.predict(X_train)
                y_test_pred_c45 = mc45.predict(X_test)

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["C4.5"],
                                                                            "scoring": ["AUC"],
                                                                            "n_leaves": [leaf_count(mc45.estimator_)],
                                                                            "max_leaves": [m],
                                                                            "regularization": ["None"],
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_c45)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_c45)]})])
                """

                # TODO: GOSDT ###
                
                ### Hierarchical shrinkage ###
                
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                cv_scores = {}
                for reg_param in reg_hs:
                    hs_skf = StratifiedKFold(n_splits=3)
                    cv_scores[reg_param] = []
                    for j, (cv_train_index, cv_val_index) in enumerate(hs_skf.split(X_train, y_train)):
                        X_cv_train, y_cv_train = X[cv_train_index, :], y[cv_train_index]
                        X_cv_val, y_cv_val = X[cv_val_index, :], y[cv_val_index]
                        hs_cv_dt = DecisionTreeClassifier(max_leaf_nodes=m)
                        hs_cv_dt.fit(X_cv_train, y_cv_train)
                        hs_cv_dt = imodels.HSTreeClassifier(hs_cv_dt, reg_param=reg_param)
                        y_val_pred = hs_cv_dt.predict(X_cv_val)
                        cv_scores[reg_param].append(roc_auc_score(y_cv_val, y_val_pred))
                cv_scores = {reg_param: np.mean(cv_scores[reg_param]) for reg_param in cv_scores.keys()}
                best_score = np.max([cv_scores[reg_param] for reg_param in cv_scores.keys()])
                best_param = [reg_param for reg_param in cv_scores.keys() if cv_scores[reg_param] == best_score][0]
                hs_reg_param = best_param
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()

                # evaluation of improvements offered by hierarchical shrinkage model
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                mshrunk = imodels.HSTreeClassifier(deepcopy(m1), reg_param=hs_reg_param) #.fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_shrunk = mshrunk.predict(X_train)
                y_test_pred_shrunk = mshrunk.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["HS (CART)"],
                                                                            "scoring": ["AUC"],
                                                                            "n_leaves": [leaf_count(mshrunk.estimator_)],
                                                                            "max_leaves": [m],
                                                                            "regularization": [hs_reg_param],
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_shrunk)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_shrunk)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])


                ### Hierarchical shrinkage (CCP) ###
            
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                cv_scores = {}
                for reg_param in reg_hs:
                    hs_skf = StratifiedKFold(n_splits=3)
                    cv_scores[reg_param] = []
                    for j, (cv_train_index, cv_val_index) in enumerate(hs_skf.split(X_train, y_train)):
                        X_cv_train, y_cv_train = X[cv_train_index, :], y[cv_train_index]
                        X_cv_val, y_cv_val = X[cv_val_index, :], y[cv_val_index]
                        hs_cv_ccp = DecisionTreeClassifier(max_leaf_nodes=m, ccp_alpha=pick_alpha(X_cv_train, y_cv_train, m, DecisionTreeClassifier))
                        hs_cv_ccp.fit(X_cv_train, y_cv_train)
                        hs_cv_ccp = imodels.HSTreeClassifier(hs_cv_ccp, reg_param=reg_param)
                        y_val_pred = hs_cv_dt.predict(X_cv_val)
                        cv_scores[reg_param].append(roc_auc_score(y_cv_val, y_val_pred))
                cv_scores = {reg_param: np.mean(cv_scores[reg_param]) for reg_param in cv_scores.keys()}
                best_score = np.max([cv_scores[reg_param] for reg_param in cv_scores.keys()])
                best_param = [reg_param for reg_param in cv_scores.keys() if cv_scores[reg_param] == best_score][0]
                hs_reg_param = best_param
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()

                # evaluation of improvements offered by hierarchical shrinkage model
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                mshrunk = imodels.HSTreeClassifier(deepcopy(mccp), reg_param=hs_reg_param) #.fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_shrunk = mshrunk.predict(X_train)
                y_test_pred_shrunk = mshrunk.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                classification_results = pd.concat([classification_results, pd.DataFrame({"task": ["classification"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["HS (CART-CCP)"],
                                                                            "scoring": ["AUC"],
                                                                            "n_leaves": [leaf_count(mshrunk.estimator_)],
                                                                            "max_leaves": [m],
                                                                            "regularization": [hs_reg_param],
                                                                            "train_score": [roc_auc_score(y_train, y_train_pred_shrunk)],
                                                                            "test_score": [roc_auc_score(y_test, y_test_pred_shrunk)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])


                classification_results.to_csv("claim_1_1_classification.csv", index = False)

## 4.2 B) Regression

In [ ]:
# number of leafs used in paper
num_of_leaves = [2, 4, 8, 12, 15, 20, 24, 28, 30, 32]
# reuglarization parameter
reg_hs = [0.1, 1.0, 10.0, 25.0, 50.0, 100.0]

# Potential problem HS appeared to be choosen via CV (hopefully they split the dataset before hand)
NUM_OF_BOOTSTRAP_SAMPS = 5
regression_results = pd.DataFrame(columns = ["task", "dataset", "boot_iter", "algorithm", "scoring", "n_leaves", "max_leaves", "regularization", "train_score", "test_score", \
                                                 "train_wall_time", "test_wall_time", "train_cpu_time", "test_cpu_time", "tunning_wall_time", "tunning_cpu_time"])

for task in DATASETS_REGRESSION:
    for samp in range(NUM_OF_BOOTSTRAP_SAMPS):
        skf = KFold(n_splits=3)
        X, y = np.array(tasks_regression[task[0]].drop("label", axis = 1)), np.array(tasks_regression[task[0]]["label"])
        for i, (train_index, test_index) in enumerate(skf.split(tasks_regression[task[0]], tasks_regression[task[0]]["label"])):
            print(f"Dataset: {task[0]}, Sample: {samp}, Fold {i}", end = "\r")

            X_train, y_train = X[train_index, :], y[train_index]
            X_test, y_test = X[test_index, :], y[test_index]

            for m in num_of_leaves:
                ### CART ###
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                m1 = DecisionTreeRegressor(max_leaf_nodes=m).fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_dt = m1.predict(X_train)
                y_test_pred_dt = m1.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                regression_results = pd.concat([regression_results, pd.DataFrame({"task": ["regression"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["DT"],
                                                                            "scoring": ["R2"],
                                                                            "n_leaves": [leaf_count(m1)],
                                                                            "max_leaves": [m],
                                                                            "regularization": ["None"],
                                                                            "train_score": [r2_score(y_train, y_train_pred_dt)],
                                                                            "test_score": [r2_score(y_test, y_test_pred_dt)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [None], 
                                                                            "tunning_cpu_time": [None]})])

                ### CART with CCP ###
                
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                best_alpha = pick_alpha(X_train, y_train, m, DecisionTreeRegressor)
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                mccp = DecisionTreeRegressor(ccp_alpha=best_alpha).fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_ccp = mccp.predict(X_train)
                y_test_pred_ccp = mccp.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                regression_results = pd.concat([regression_results, pd.DataFrame({"task": ["regression"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["CCP"],
                                                                            "scoring": ["R2"],
                                                                            "n_leaves": [leaf_count(mccp)],
                                                                            "max_leaves": [m],
                                                                            "regularization": [best_alpha],
                                                                            "train_score": [r2_score(y_train, y_train_pred_ccp)],
                                                                            "test_score": [r2_score(y_test, y_test_pred_ccp)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])

                ### C4.5 DT ###
                """
                mc45 = imodels.C45TreeClassifier().fit(X_train, y_train)
                y_train_pred_c45 = mc45.predict(X_train)
                y_test_pred_c45 = mc45.predict(X_test)

                regression_results = pd.concat([regression_results, pd.DataFrame({"task": ["regression"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["C4.5"],
                                                                            "scoring": ["R2"],
                                                                            "n_leaves": [leaf_count(mc45.estimator_)],
                                                                            "max_leaves": [m],
                                                                            "regularization": ["None"],
                                                                            "train_score": [r2_score(y_train, y_train_pred_c45)],
                                                                            "test_score": [r2_score(y_test, y_test_pred_c45)]})])
                """

                # TODO: GOSDT ###
                
                ### Hierarchical shrinkage ###
                
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                cv_scores = {}
                for reg_param in reg_hs:
                    hs_skf = KFold(n_splits=3)
                    cv_scores[reg_param] = []
                    for j, (cv_train_index, cv_val_index) in enumerate(hs_skf.split(X_train, y_train)):
                        X_cv_train, y_cv_train = X[cv_train_index, :], y[cv_train_index]
                        X_cv_val, y_cv_val = X[cv_val_index, :], y[cv_val_index]
                        hs_cv_dt = DecisionTreeRegressor(max_leaf_nodes=m)
                        hs_cv_dt.fit(X_cv_train, y_cv_train)
                        hs_cv_dt = imodels.HSTreeRegressor(hs_cv_dt, reg_param=reg_param)
                        y_val_pred = hs_cv_dt.predict(X_cv_val)
                        cv_scores[reg_param].append(r2_score(y_cv_val, y_val_pred))
                cv_scores = {reg_param: np.mean(cv_scores[reg_param]) for reg_param in cv_scores.keys()}
                best_score = np.max([cv_scores[reg_param] for reg_param in cv_scores.keys()])
                best_param = [reg_param for reg_param in cv_scores.keys() if cv_scores[reg_param] == best_score][0]
                hs_reg_param = best_param
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()

                # evaluation of improvements offered by hierarchical shrinkage model
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                mshrunk = imodels.HSTreeRegressor(deepcopy(m1), reg_param=hs_reg_param) #.fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_shrunk = mshrunk.predict(X_train)
                y_test_pred_shrunk = mshrunk.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                regression_results = pd.concat([regression_results, pd.DataFrame({"task": ["regression"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["HS (CART)"],
                                                                            "scoring": ["R2"],
                                                                            "n_leaves": [leaf_count(mshrunk.estimator_)],
                                                                            "max_leaves": [m],
                                                                            "regularization": [hs_reg_param],
                                                                            "train_score": [r2_score(y_train, y_train_pred_shrunk)],
                                                                            "test_score": [r2_score(y_test, y_test_pred_shrunk)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])


                ### Hierarchical shrinkage (CCP) ###
            
                # measure tunning time
                start_wall_time_tunning = time.time()
                start_cpu_time_tunning = time.process_time()
                
                cv_scores = {}
                for reg_param in reg_hs:
                    hs_skf = KFold(n_splits=3)
                    cv_scores[reg_param] = []
                    for j, (cv_train_index, cv_val_index) in enumerate(hs_skf.split(X_train, y_train)):
                        X_cv_train, y_cv_train = X[cv_train_index, :], y[cv_train_index]
                        X_cv_val, y_cv_val = X[cv_val_index, :], y[cv_val_index]
                        hs_cv_ccp = DecisionTreeRegressor(max_leaf_nodes=m, ccp_alpha=pick_alpha(X_cv_train, y_cv_train, m, DecisionTreeRegressor))
                        hs_cv_ccp.fit(X_cv_train, y_cv_train)
                        hs_cv_ccp = imodels.HSTreeRegressor(hs_cv_ccp, reg_param=reg_param)
                        y_val_pred = hs_cv_dt.predict(X_cv_val)
                        cv_scores[reg_param].append(r2_score(y_cv_val, y_val_pred))
                cv_scores = {reg_param: np.mean(cv_scores[reg_param]) for reg_param in cv_scores.keys()}
                best_score = np.max([cv_scores[reg_param] for reg_param in cv_scores.keys()])
                best_param = [reg_param for reg_param in cv_scores.keys() if cv_scores[reg_param] == best_score][0]
                hs_reg_param = best_param
                
                end_wall_time_tunning = time.time()
                end_cpu_time_tunning = time.process_time()

                # evaluation of improvements offered by hierarchical shrinkage model
                
                # measure train time
                start_wall_time_train = time.time()
                start_cpu_time_train = time.process_time()
                
                mshrunk = imodels.HSTreeRegressor(deepcopy(mccp), reg_param=hs_reg_param) #.fit(X_train, y_train)
                
                end_wall_time_train = time.time()
                end_cpu_time_train = time.process_time()
                
                # measure test time
                start_wall_time_test = time.time()
                start_cpu_time_test = time.process_time()
                
                y_train_pred_shrunk = mshrunk.predict(X_train)
                y_test_pred_shrunk = mshrunk.predict(X_test)
                
                end_wall_time_test = time.time()
                end_cpu_time_test = time.process_time()

                regression_results = pd.concat([regression_results, pd.DataFrame({"task": ["regression"], 
                                                                            "dataset": [task[0]],
                                                                            "boot_iter": [samp],
                                                                            "algorithm": ["HS (CART-CCP)"],
                                                                            "scoring": ["R2"],
                                                                            "n_leaves": [leaf_count(mshrunk.estimator_)],
                                                                            "max_leaves": [m],
                                                                            "regularization": [hs_reg_param],
                                                                            "train_score": [r2_score(y_train, y_train_pred_shrunk)],
                                                                            "test_score": [r2_score(y_test, y_test_pred_shrunk)],
                                                                            "train_wall_time": [end_wall_time_train - start_wall_time_train],
                                                                            "test_wall_time": [end_wall_time_test - start_wall_time_test],
                                                                            "train_cpu_time": [end_cpu_time_train - start_cpu_time_train],
                                                                            "test_cpu_time": [end_cpu_time_test - start_cpu_time_test],
                                                                            "tunning_wall_time": [end_wall_time_tunning - start_wall_time_tunning], 
                                                                            "tunning_cpu_time": [end_cpu_time_tunning - start_cpu_time_tunning]})])


                regression_results.to_csv("claim_1_1_regression.csv", index = False)